In [53]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib inline
import tensorflow as tf
import keras
import glob
import cv2
import pickle, datetime

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import LSTM, Input, TimeDistributed,Convolution2D,Activation
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import RMSprop, SGD
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model

from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Import the backend
import tensorflow.keras.backend as K
import os

### Read the train & test Images and preprocessing

In [54]:
train_fruit_images = []
train_fruit_labels = [] 
for directory_path in glob.glob("fruits-360-original-size/Training/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (227, 227))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_fruit_images.append(img)
        train_fruit_labels.append(fruit_label)
train_fruit_images = np.array(train_fruit_images)
train_fruit_labels = np.array(train_fruit_labels)

In [55]:
label_to_id = {v:i for i,v in enumerate(np.unique(train_fruit_labels))}
id_to_label = {v: k for k, v in label_to_id.items()}
train_label_ids = np.array([label_to_id[x] for x in train_fruit_labels])

In [56]:
train_fruit_images.shape, train_label_ids.shape, train_fruit_labels.shape

((1091, 227, 227, 3), (1091,), (1091,))

In [57]:
# test
test_fruit_images = []
test_fruit_labels = [] 
for directory_path in glob.glob("fruits-360-original-size/Test/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        img = cv2.resize(img, (227, 227))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        test_fruit_images.append(img)
        test_fruit_labels.append(fruit_label)
test_fruit_images = np.array(test_fruit_images)
test_fruit_labels = np.array(test_fruit_labels)

In [58]:
test_label_ids = np.array([label_to_id[x] for x in test_fruit_labels])

In [59]:
test_fruit_images.shape, test_label_ids.shape

((543, 227, 227, 3), (543,))

In [60]:
x_train, y_train, x_test, y_test, N_CATEGORY =train_fruit_images,train_fruit_labels,test_fruit_images,test_fruit_labels,len(label_to_id)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, N_CATEGORY)

(1091, 227, 227, 3) (1091,) (543, 227, 227, 3) (543,) 6


### Classes with its ids

In [61]:
id_to_label

{0: 'apple_golden_1',
 1: 'cabbage_white_1',
 2: 'carrot_1',
 3: 'cucumber_1',
 4: 'pear_1',
 5: 'zucchini_1'}

###  CNN Architecture

In [62]:
def get_alexnet(input_shape,nb_classes): 
    model = Sequential()

    # Layer 1
    model.add(Convolution2D(96, 11, 11, input_shape = input_shape, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2
    model.add(Convolution2D(128, 5, 5, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(384, 3, 3, padding='same'))
    model.add(Activation('relu'))

    # Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(192, 3, 3, padding='same'))
    model.add(Activation('relu'))

    # Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, padding='same'))
    model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 6
    model.add(GlobalAveragePooling2D())
    model.add(Dense(4096, kernel_initializer ='glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Layer 7
    model.add(Dense(4096, kernel_initializer ='glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # Layer 8
    model.add(Dense(nb_classes, kernel_initializer ='glorot_normal'))
    model.add(Activation('tanh'))
    return model

In [63]:
alexnet = get_alexnet((227,227,3),N_CATEGORY)
alexnet.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 21, 21, 96)        34944     
                                                                 
 activation_18 (Activation)  (None, 21, 21, 96)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 10, 10, 96)       0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 2, 2, 128)         307328    
                                                                 
 activation_19 (Activation)  (None, 2, 2, 128)         0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 1, 1, 128)        0         
 2D)                                                  

In [64]:
#Normalization of the images and one-hot encoding of the labels
alexnet.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics=['accuracy'])
X_normalized = np.array(x_train / 255.0 - 0.5 )
X_normalized_test = np.array(x_test / 255.0 - 0.5 )

label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)
y_one_hot_test = label_binarizer.fit_transform(y_test)

In [65]:
#Training the AlexNet model with the normalized image data and labels
alexnet.fit(X_normalized, y_one_hot, batch_size=5, epochs=2,verbose=1, validation_data=[X_normalized_test,y_one_hot_test])

Epoch 1/2
219/219 [==============================] - 57s 258ms/step - loss: nan - accuracy: 0.2686 - val_loss: nan - val_accuracy: 0.2836
Epoch 2/2
219/219 [==============================] - 57s 259ms/step - loss: nan - accuracy: 0.2823 - val_loss: nan - val_accuracy: 0.2836


#### Result of the Alexnet
The accuracy of the whole AlexNet model is 0.2686 which is 26.86%.

Here, the classification is performed by the other Fully-connected layer and Softmax function.


In [66]:
#Save the AlexNet model for the future use(as it takes hours to be trained!)
alexnet.save('alexnetfruit.h5') 

### Feature Extraction by CNN

In [68]:
#Pick the first Fully-Connected layer as the features which will be of dimension (1 x 4096)
layer_name = 'dense_3'
FC_layer_model = Model(inputs=alexnet.input,
                                 outputs=alexnet.get_layer(layer_name).output)

In [69]:
#Find the Features for n number of train images and we will get n x 4096
#This means we will get 4096 features for each images.
i=0
features=np.zeros(shape=(x_train.shape[0],4096))
for directory_path in glob.glob("fruits-360-original-size/Training/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)    
        img = cv2.resize(img, (227, 227))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = np.expand_dims(img, axis=0)
        FC_output = FC_layer_model.predict(img)
        features[i]=FC_output
        i+=1


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 37ms/step


In [70]:
#Save the features of the train images to use it in future.
np.save('features', features)

In [71]:
#Name the feature rows as f_0, f_1, f_2...
feature_col=[]
for i in range(4096):
    feature_col.append("f_"+str(i))
    i+=1
    

In [72]:
#Create DataFrame with features and coloumn name
train_features=pd.DataFrame(data=features,columns=feature_col)
feature_col = np.array(feature_col)

train_class = list(np.unique(train_label_ids))
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_label_ids.shape)
train_class

Training Features Shape: (1091, 4096)
Training Labels Shape: (1091,)


[0, 1, 2, 3, 4, 5]

### Testing the  Algorithm

#### Feature Extraction by CNN (AlexNet)

In [75]:
#Find the Features from Alexnet's FC layer for n number of test images and we will get n x 4096
i=0
features_test=np.zeros(shape=(y_test.shape[0],4096))
for directory_path in glob.glob("fruits-360-original-size/Test/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)    
        img = cv2.resize(img, (227, 227))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = np.expand_dims(img, axis=0)
        FC_output = FC_layer_model.predict(img)
        features_test[i]=FC_output
        i+=1


1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 37ms/step


In [76]:
#Create DataFrame with features and coloumn name
test_features=pd.DataFrame(data=features_test,columns=feature_col)
feature_col = np.array(feature_col)

print('Test Features Shape:', test_features.shape)
print('Test Labels Shape:', test_label_ids.shape)


Test Features Shape: (543, 4096)
Test Labels Shape: (543,)
